In [10]:
import pandas as pd
from string import punctuation
import re
import numpy as np
from setup import *
import glob
import os
import pickle
from collections import Counter
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize, TweetTokenizer
from nltk.corpus import stopwords
from matplotlib import pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence



# The Process

## [I. Data Acquisition](#data_acquisition_link)

## [II. Preprocessing](#preprocessing_link)

## Save Preprocessing Steps
1. Save teh Tf-Idf using pickle
1. Condense the preprocessing step into a function

## Model Setup
1. Support Vector Machine Regression
1. test-train split

## Train The Model, Tune Hyperparameters
1. Gridsearch on some parameters (maybe include how much reach to give to model)
1. Save all of the models, compare outputs, etc
1. Cross validate

## Test Model
1. Test each model on the testing set, look for best results. 

## Save Best Model 
1. Use pickle to save the best model for future use. 

<a id='data_acquisition_link'></a>
# Data Acquisition
1. Check if Files have been collected already
1. Use Glob to collect all of the files in the Data Folder
1. Save to single file for later use

In [11]:

if os.path.isfile('All_Files.csv'):
    df = pd.read_csv('All_Files.csv')

else:
    file_list = glob.glob("./data/Facebook Insights*")

    df_list = []

    for file in file_list:
        df_list.append(pd.read_csv(file,encoding='iso-8859-1').loc[1:])

    # for df in df_list:
    #     display(df)

    df = pd.concat(df_list).reset_index()
    df.to_csv('All_Files.csv')
display(df)

,Unnamed: 0,index,Post ID,Permalink,Post Message,Type,Countries,Languages,Posted,Audience Targeting,...,Lifetime Organic views to 95%,Lifetime Organic views to 95%.1,Lifetime Paid views to 95%,Lifetime Paid views to 95%.1,Lifetime Organic Video Views,Lifetime Organic Video Views.1,Lifetime Paid Video Views,Lifetime Paid Video Views.1,Lifetime Average time video viewed,Lifetime Video length
0,0,1,171162119680585_859052127558244,https://www.facebook.com/TarletonCOGS/posts/85...,Tomorrow is the COGS fall open house at Tarlet...,Link,nan,nan,9/30/16 4:26 PM,,...,0,0,0,0,0,0,0,0,0,0
1,1,2,171162119680585_859037424226381,https://www.facebook.com/TarletonCOGS/posts/85...,It's not too late to register for Saturday's O...,Link,nan,nan,9/29/16 4:11 PM,,...,0,0,0,0,0,0,0,0,0,0
2,2,3,171162119680585_859081790888611,https://www.facebook.com/TarletonCOGS/posts/85...,Tarleton's Fall Job Fair is being held October...,Link,nan,nan,9/29/16 6:19 AM,,...,0,0,0,0,0,0,0,0,0,0
3,3,4,171162119680585_859035847559872,https://www.facebook.com/TarletonCOGS/posts/85...,Questions about the #GRE? Attend this Saturda...,Link,nan,nan,9/28/16 4:07 PM,,...,0,0,0,0,0,0,0,0,0,0
4,4,5,171162119680585_859091770887613,https://www.facebook.com/TarletonCOGS/posts/85...,The deadline to register for the #TAMUSPathway...,Link,nan,nan,9/28/16 9:40 AM,,...,0,0,0,0,0,0,0,0,0,0
5,5,6,171162119680585_859074730889317,https://www.facebook.com/TarletonCOGS/posts/85...,Access the services of The Writing Center at ...,Link,nan,nan,9/28/16 6:02 AM,,...,0,0,0,0,0,0,0,0,0,0
6,6,7,171162119680585_859035654226558,https://www.facebook.com/TarletonCOGS/posts/85...,Learn more about graduate programs at Tarleton...,Link,nan,nan,9/27/16 4:07 PM,,...,0,0,0,0,0,0,0,0,0,0
7,7,8,171162119680585_859058130890977,https://www.facebook.com/TarletonCOGS/posts/85...,Looking to further your career as an education...,Photo,nan,nan,9/27/16 6:43 AM,,...,0,0,0,0,0,0,0,0,0,0
8,8,9,171162119680585_859028140893976,https://www.facebook.com/TarletonCOGS/posts/85...,Did you miss Tarleton on Tour on Saturday? Yo...,Link,nan,nan,9/26/16 4:53 PM,,...,0,0,0,0,0,0,0,0,0,0
9,9,10,171162119680585_859018744228249,https://www.facebook.com/TarletonCOGS/posts/85...,Thanks to all who came by the Arlington Conven...,Photo,nan,nan,9/26/16 7:17 AM,,...,0,0,0,0,0,0,0,0,0,0


<a id='preprocessing_link'></a>
# Preprocessing

1. Collect the columns for reach, engagement
1. Dropna, since they will not contribute to finding good posts
1. Create proportion column
1. Make Variable for how much reach to keep
1. Tokenize
1. Tf-Idf


Use This next cell to restart the fresh df, to check changes in the preprocessing

In [18]:
df = pd.read_csv('All_Files.csv')
df = df[['Post Message', 'Type', 'Posted', 'Lifetime Post Total Reach','Lifetime Engaged Users']]
df.columns = ['Post', 'Type','Date','Reach','Engaged']


In [19]:
df.dropna(inplace=True)
ndf = pd.merge(df.groupby('Post')['Engaged'].apply(lambda p: max(p)).reset_index(),df[['Post', 'Type', 'Date', 'Reach', 'Engaged']], on = ['Post','Engaged'], how = 'inner')
ndf.drop_duplicates('Post',inplace=True)
fdf = ndf.loc[ndf['Reach']>=50].reset_index(drop=True)
display(fdf)
fdf['Prop'] = fdf['Engaged']/fdf['Reach']

,Post,Engaged,Type,Date,Reach
0,"""An education is just another tool we can use ...",6,Status,8/9/16 5:03 PM,174
1,"""Despite his brilliance, Einstein couldn't sec...",5,Link,3/16/16 6:47 AM,227
2,"""Get off the grass!"" Check out some Tarleton ...",172,Link,2/11/16 10:05 AM,2796
3,"""Healers need healers!"" See what Dr. Merriman ...",4,Link,5/18/16 4:18 PM,150
4,"""I know who I am. I know what I?m capable of.""...",3,Link,2/8/16 9:17 AM,153
5,"""It does not matter how slowly you go as long ...",7,Photo,6/6/16 5:38 PM,243
6,"""My degree is the gateway to __."" Watch our n...",15,Link,7/18/16 5:41 AM,328
7,"""Tarleton College"" in 1899. https://www.facebo...",24,Photo,5/19/16 6:22 AM,426
8,"""The way to get started is to quit talking and...",10,Link,8/15/16 8:31 AM,185
9,"""Then, in that hour of deliverance, my heart s...",13,Photo,5/29/17 6:01 AM,726


In [ ]:
keep_reach = 50